In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import os
def fcn_layer(inputs,
             input_dim,
             output_dim,
             activation=None):
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
    b = tf.Variable(tf.zeros([output_dim]))
    XWB = tf.matmul(inputs, W) + b
    if activation == None:
        outputs = XWB
    else:
        outputs = activation(XWB)
    return outputs

E:\python\python_install\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

H1_NN = 256
H2_NN = 32

x = tf.placeholder(tf.float32, [None, 784], name="X")
y = tf.placeholder(tf.float32, [None, 10], name="Y")

h1 = fcn_layer(inputs=x,
              input_dim=784,
              output_dim=H1_NN,
              activation=tf.nn.relu)
h2 = fcn_layer(inputs=h1,
              input_dim=H1_NN,
              output_dim=H2_NN,
              activation=tf.nn.relu)
forward = fcn_layer(inputs=h2,
                   input_dim=H2_NN,
                   output_dim=10)
pred = tf.nn.softmax(forward)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
loss_function = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=forward, labels=y))
correct_prediction = tf.equal(tf.arg_max(pred, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Use `argmax` instead


In [4]:
ckpt_dir = './ckpt_dir/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [5]:
saver = tf.train.Saver()

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


ckpt = tf.train.get_checkpoint_state(ckpt_dir)
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)


INFO:tensorflow:Restoring parameters from ./ckpt_dir/mnist_h256_h32_model.ckpt


In [6]:
print("Accuracy:", accuracy.eval(session=sess,
                                feed_dict={x: mnist.test.images, y: mnist.test.labels}))

Accuracy: 0.9701


In [7]:
train_epochs = 30
batch_size = 50
total_batch = int(mnist.train.num_examples / batch_size)
display_step = 2
learning_rate = 0.01
save_step = 5

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

In [9]:
image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.image('input', image_shaped_input, 10)
tf.summary.histogram('forward', forward)
tf.summary.scalar('loss', loss_function)
tf.summary.scalar('accuracy', accuracy)
merged_summary_op = tf.summary.merge_all()
writer = tf.summary.FileWriter('log/', sess.graph)

In [10]:
from time import time

startTime = time()
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x:xs, y:ys})
        summary_str = sess.run(merged_summary_op, feed_dict={x:xs, y:ys})
        writer.add_summary(summary_str, epoch)
        
    loss, acc = sess.run([loss_function, accuracy],
                        feed_dict={x:mnist.validation.images,
                                  y:mnist.validation.labels})
    if (epoch+1) % display_step == 0:
        print("Train Epoch %02d: Loss=%.9f Accuracy=%.4f" % (epoch+1, loss, acc))
        
    if (epoch+1) % save_step == 0:
        saver.save(sess, os.path.join(ckpt_dir, 'mnist_h256_h32_model_{:06d}.ckpt'.format(epoch+1)))
        print('mnist_h256_h32_model_{:06d}.ckpt saved'.format(epoch+1))
saver.save(sess, os.path.join(ckpt_dir,'mnist_h256_h32_model.ckpt'))
print("Model saved!")
           
duration = time() - startTime
print("Train Finished takes:%.2f" % duration)

Train Epoch 02: Loss=0.130917773 Accuracy=0.9668
Train Epoch 04: Loss=0.122820973 Accuracy=0.9672
mnist_h256_h32_model_000005.ckpt saved
Train Epoch 06: Loss=0.151775658 Accuracy=0.9688
Train Epoch 08: Loss=0.174026921 Accuracy=0.9662
Train Epoch 10: Loss=0.173033625 Accuracy=0.9700
mnist_h256_h32_model_000010.ckpt saved
Train Epoch 12: Loss=0.156657591 Accuracy=0.9702
Train Epoch 14: Loss=0.142144710 Accuracy=0.9754
mnist_h256_h32_model_000015.ckpt saved
Train Epoch 16: Loss=0.175115019 Accuracy=0.9718
Train Epoch 18: Loss=0.152758017 Accuracy=0.9726
Train Epoch 20: Loss=0.264420480 Accuracy=0.9674
mnist_h256_h32_model_000020.ckpt saved
Train Epoch 22: Loss=0.245184124 Accuracy=0.9704
Train Epoch 24: Loss=0.214590997 Accuracy=0.9740
mnist_h256_h32_model_000025.ckpt saved
Train Epoch 26: Loss=0.253699601 Accuracy=0.9760
Train Epoch 28: Loss=0.317332357 Accuracy=0.9734
Train Epoch 30: Loss=0.292354733 Accuracy=0.9746
mnist_h256_h32_model_000030.ckpt saved
Model saved!
Train Finished tak

In [11]:
accu_test = sess.run(accuracy, feed_dict={x: mnist.test.images,
                                          y: mnist.test.labels})
print("Test Accuracy:", accu_test)

Test Accuracy: 0.9741
